In [18]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

data = pd.read_excel("./data/cda1_1.xlsx")
data.head(5)

,主页ID,主页名称,分类,message,name,description,发文类型,y
0,155000000000000,纬来体育台官方粉丝团,电视频道,中华队长林智胜 三分砲！！！\n人如其名 真的「致胜」阿～,NaN,NaN,video,H
1,359000000000,公视粉丝团,电视频道,您所选择的电视频道，将会决定您的未来~\n遥控器的决定权，就在你手上！\n\n#看见更好的未来,2015公视电视募款 公视让你看见更好的未来,这是一个资讯爆炸的时代，你所选择的资讯将决定你的视野；你所选择的电视频道，将会决定你的未来 ...,video,L
2,359000000000,公视粉丝团,电视频道,好演员值得更多肯定和掌声！\n\n吴慷仁提到大概有六、七年资历的演员，所面临的最大困境，是赚...,公视粉丝团,NaN,link,L
3,190000000000,Mobile01,网路社群,大师兄 一发炸裂！！帅啊啊啊啊啊啊啊！！！,来自 Mobile01 贴文的相片,NaN,photo,H
4,359000000000,公视粉丝团,电视频道,「再不救，就来不及了！如果没有很强力的救，再三十年，你可能找不到鹿港的味道了！」\n\n「两...,【我们的岛】抢救鹿港,叶明兰、叶镇中 / 采访报导 「再不救，就来不及了！如果没有很强力的救，再三十年，你可能找不...,link,L


In [19]:
X = data[data.columns[:-1]]
y = data['y']

In [20]:
# 这段代码记得删除了
def column_mapping_str(X, col_name='', prefix_name='', begin_idx=0):
    """
    根据指定列的值，生成mapping字符串
    """
    col_value_counts = X[col_name].value_counts()
    print("------------%s每个项的分布--------------" % col_name)
    print(col_value_counts)
    print("--------------------------------------")
    label_str = ""
    for idx, key in enumerate(col_value_counts.keys()):
        label_str = "".join([label_str, "'", str(key), "'", ": ", str(idx + begin_idx), ", "])
    label_str = label_str[0:-2]
    print("mapping_%s = {%s}" % (prefix_name, label_str))
    print("X['%s'] = X['%s'].map(mapping_%s).fillna(-99).astype(int)" % (col_name, col_name, prefix_name))
    print("X_%s = pd.get_dummies(X['%s'], prefix='%s', drop_first=True)" % (prefix_name, col_name, prefix_name))
    print("X = pd.concat([X, pd.DataFrame(X_%s, index=X.index)], axis=1)" % prefix_name)
    print("X = X.drop(labels='%s', axis=1)" % col_name)
    print("--------------------------------------")

column_mapping_str(X, col_name='分类', prefix_name='fl', begin_idx=0)

------------分类每个项的分布--------------
新闻        4807
电视频道       935
书籍杂志       891
节目         347
电视剧        261
媒体其他       252
网路社群       232
电影公司        62
电影          29
广播           5
音乐           4
电影相关活动       4
Name: 分类, dtype: int64
--------------------------------------
mapping_fl = {'新闻': 0, '电视频道': 1, '书籍杂志': 2, '节目': 3, '电视剧': 4, '媒体其他': 5, '网路社群': 6, '电影公司': 7, '电影': 8, '广播': 9, '音乐': 10, '电影相关活动': 11}
X['分类'] = X['分类'].map(mapping_fl).fillna(-99).astype(int)
X_fl = pd.get_dummies(X['分类'], prefix='fl', drop_first=True)
X = pd.concat([X, pd.DataFrame(X_fl, index=X.index)], axis=1)
X = X.drop(labels='分类', axis=1)
--------------------------------------


In [21]:
mapping_fl = {'电视频道': 0, '网路社群': 1, '新闻': 2, '书籍杂志': 3, '电视剧': 4, '电影公司': 5, '节目': 6, '媒体其他': 7, '电影': 5, '音乐': 7, '电影相关活动': 5, '广播': 7}
X['分类'] = X['分类'].map(mapping_fl).fillna(7).astype(int)
X_fl = pd.get_dummies(X['分类'], prefix='fl', drop_first=True)
X = pd.concat([X, pd.DataFrame(X_fl, index=X.index)], axis=1)
X = X.drop(labels='分类', axis=1)

In [22]:
mapping_fwlx = {'link': 1, 'photo': 2, 'video': 3}
X['发文类型'] = X['发文类型'].map(mapping_fwlx).fillna(1).astype(int)
X_fwlx = pd.get_dummies(X['发文类型'], prefix='fwlx', drop_first=True)
X = pd.concat([X, pd.DataFrame(X_fwlx, index=X.index)], axis=1)
X = X.drop(labels='发文类型', axis=1)

In [23]:
from nlp import *
stop_words = get_stop_words('./data/stop_words.txt', encoding='utf-8')
print(split_word("「呸姊」蔡依林2日晚间出席南韩年度盛事MAMA舞台",
                 stop_words, ['蔡依林', '叶明兰','叶镇中', '宋家豪', '白先勇','林智胜']))

total counts:  1913
姊 蔡依林 日 晚间 出席 南韩 年度 盛事 舞台


In [24]:
X_docs = X.apply(lambda x: split_word(
    str(x['message']) + str(x['name']) + str(x['description']),
    stop_words,
    ['蔡依林', '叶明兰','叶镇中', '宋家豪', '白先勇','林智胜']),
    axis=1)
X_docs

0                                     中华 队长 林智胜 三分 砲 名 致胜
1       选择 电视频道 将会 未来 遥控器 决定权 手上 更好 未来 公视 电视 募款 公视 更好 ...
2       演员 值得 肯定 掌声 吴慷仁 提到 七年 资历 演员 面临 困境 赚 不到 钱 去年 中国...
3                                师兄 一发 炸裂 帅 啊啊啊 啊啊啊 贴文 相片
4       不救 强力 救 三十年 找 不到 鹿港 味道 两百年 老房子 消失 速度 月 一栋 拆 一府...
                              ...                        
7824    镶上 绿色 宝石 番薯 好美 土星 中天 新闻 高画质 直播 网址 直播 时段 早上 点 凌...
7825    早鸟 福利 老师 唯美 瞬间 捕捉到 光棍节 告诉 单身 朋友 孤单 注 深夜 福利 早鸟 ...
7826    佩颖 主任 自暴自弃 摆脱 烂 桃花 新招 每周五 晚间 十点 三立 台 每周六 晚间 八点...
7827    合 作 小姐 蔡依林 蔡依林 后台 照曝光 甜呼 爱心 主人 娱乐 星光 云 东森 新闻 云...
7828    联合报 一个班 大学 班级 变 家教 班 系 注册 挂零 校拉 警报 注册 率 不到 成 大...
Length: 7829, dtype: object

In [25]:
tfidf_vec, tfidf, tfidf_pca = tfidf(X_docs, min_df=10, output_dimension=50)
tfidf_vec

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-0.036074,-0.037224,0.175324,0.067177,-0.038803,-0.013186,0.000022,-0.021062,0.029179,-0.006966,...,0.003655,-0.005792,0.005349,-0.004669,0.014479,-0.004179,0.010859,-0.007530,0.001138,0.009929
1,-0.031605,-0.003449,-0.015222,-0.016508,-0.013593,0.170616,-0.014756,-0.034352,0.040438,-0.025265,...,0.010756,-0.081594,-0.001787,-0.000439,0.014923,0.065202,-0.009457,0.000964,0.037471,-0.029436
2,-0.036402,-0.039124,-0.022664,-0.014946,-0.010846,0.174594,-0.011937,-0.025843,-0.001671,0.023112,...,0.021736,-0.092286,-0.017522,-0.005094,0.034725,0.043981,0.044907,-0.077856,0.022194,0.041834
3,0.094093,-0.014604,0.011951,0.073510,-0.036283,0.014400,0.031784,-0.005984,-0.002444,0.003320,...,0.009558,-0.005377,0.058363,-0.008290,0.000545,-0.026318,0.020720,-0.007274,0.009805,0.016902
4,-0.032072,-0.017700,-0.018555,-0.008953,-0.015091,0.014936,0.008349,-0.021848,0.026676,-0.036702,...,-0.017406,-0.017543,-0.003095,-0.016092,-0.011023,0.003980,0.025369,-0.002013,-0.008569,-0.006933
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7824,-0.058843,-0.026246,0.046905,-0.013738,0.160530,0.032444,0.021212,0.185571,-0.054165,-0.025373,...,-0.066663,0.030392,0.022287,0.066310,0.026433,0.073063,-0.078954,0.120526,0.172732,0.012184
7825,0.076465,-0.010671,-0.000792,-0.052848,-0.054224,-0.008984,0.150329,-0.009142,0.008655,-0.001768,...,0.007785,-0.000455,-0.003995,-0.024385,0.007059,-0.007436,0.010025,-0.000517,-0.000594,-0.003436
7826,0.235819,0.008249,0.034160,-0.115550,-0.115495,0.060225,0.319520,0.012135,0.019982,0.036935,...,-0.004863,-0.015199,-0.003654,0.015673,0.034823,-0.000993,0.033195,0.021945,0.001406,-0.009662
7827,-0.060789,-0.028829,-0.010195,-0.003784,0.003063,-0.006041,-0.026829,0.075225,-0.149959,0.126474,...,0.009011,0.091708,0.011561,-0.131854,0.170288,-0.040744,0.051742,0.006002,-0.062201,-0.018265


In [31]:
X = X.drop(labels=['主页ID', '主页名称', 'message', 'name', 'description'], axis=1)
X

,fl_1,fl_2,fl_3,fl_4,fl_5,fl_6,fl_7,fwlx_2,fwlx_3
0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
7824,0,0,0,0,0,0,0,0,0
7825,0,0,0,1,0,0,0,1,0
7826,0,0,0,1,0,0,0,1,0
7827,0,1,0,0,0,0,0,0,0


In [27]:
y_mapping = {'L': 0, 'H': 1}
y = y.map(y_mapping)

In [33]:
train = pd.concat([X, tfidf_vec, y], axis=1)
train.to_csv("./data/1_features_1.csv")
train.head(5)

,fl_1,fl_2,fl_3,fl_4,fl_5,fl_6,fl_7,fwlx_2,fwlx_3,0,...,41,42,43,44,45,46,47,48,49,y
0,0,0,0,0,0,0,0,0,1,-0.036074,...,-0.005792,0.005349,-0.004669,0.014479,-0.004179,0.010859,-0.007530,0.001138,0.009929,1
1,0,0,0,0,0,0,0,0,1,-0.031605,...,-0.081594,-0.001787,-0.000439,0.014923,0.065202,-0.009457,0.000964,0.037471,-0.029436,0
2,0,0,0,0,0,0,0,0,0,-0.036402,...,-0.092286,-0.017522,-0.005094,0.034725,0.043981,0.044907,-0.077856,0.022194,0.041834,0
3,1,0,0,0,0,0,0,1,0,0.094093,...,-0.005377,0.058363,-0.008290,0.000545,-0.026318,0.020720,-0.007274,0.009805,0.016902,1
4,0,0,0,0,0,0,0,0,0,-0.032072,...,-0.017543,-0.003095,-0.016092,-0.011023,0.003980,0.025369,-0.002013,-0.008569,-0.006933,0
